# Keras 개발 Process

1. 입력 텐서(X)와 출력 텐서(y)로 이뤄진 **훈련 데이터를 정의**

2. 입력과 출력을 연결하는 Layer(층)으로 이뤄진 **네트워크(모델)을 정의**
    - Sequential 방식: 순서대로 쌓아올린 네트워크로 이뤄진 모델을 생성하는 방식
    - Functional API 방식: 다양한 구조의 네트워크로 이뤄진 모델을 생성하는 방식
    - Subclass 방식: 네트워크를 정의하는 클래스를 구현.
    

3. **모델 Compile(컴파일)**
    - 모델이 Train(학습)할때 사용할 손실함수(Loss Function), 최적화기법(Optimizer), 학습과정을 모니터링할 평가지표(Metrics)를 설정
    
    > Compile: 실행할 수 있는 상태로 만들어 주는 것.

4. **Training(학습/훈련)**
    - Train dataset을 이용해 모델을 Train 시킨다.

# MNIST 이미지 분류 
- ### [MNIST](https://ko.wikipedia.org/wiki/MNIST_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4)(Modified National Institute of Standards and Technology) database
- 흑백 손글씨 숫자 0-9까지 10개의 범주로 구분해놓은 데이터셋
- 하나의 이미지는 28 * 28 pixel 의 크기
- 6만개의 Train 이미지와 1만개의 Test 이미지로 구성됨.

##### import

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
# seed값 설정
np.random.seed(0)
tf.random.set_seed(0)
random.seed(0)

##### MNIST dataset Loading

In [ ]:
(train_image, train_label), (test_image, test_label) = keras.datasets.mnist.load_data()

In [ ]:
train_image.shape, train_label.shape, test_image.shape, test_label.shape

In [ ]:
# X값-image 확인
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))

for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(train_image[i], cmap='gray')
    plt.title(f'{train_label[i]}', fontsize=20)
    plt.axis('off')

plt.tight_layout()
plt.show()

## 데이터 준비
- X (Input Data Image)
    - 0 ~ 1 사이의 값으로 정규화 시킨다.
- y (Output Data)
    - one hot encoding 처리
        - Label이 다중분류(Multi class classification)일 경우 One Hot Encoding 한다.
            - 딥러닝 모델이 각 Label별 확률을 출력하도록 정의 되기 때문에 y(정답)도 같은 형태로 만들어 줘야 한다.
    - Keras의 onehot encoding 처리 함수
        - tensorflow.keras.utils.to_categorical()

##### input image(X)를 정규화. 
- 이미지를 0 ~ 1 로 정규화 하고 타입을 float32로 변환

In [ ]:
X_train = train_image.astype("float32")/255
X_test = test_image.astype('float32')/255

##### label(y)를 one hot encoding

In [ ]:
np.unique(train_label, return_counts=True)

In [ ]:
y_train = keras.utils.to_categorical(train_label, num_classes=10)
y_test = keras.utils.to_categorical(test_label)

y_train.shape, y_test.shape

## 네트워크(모델) 정의
- Network : 전체 모델 구조

In [ ]:
# Sequential model 정의 2: 모델객체를 생성하고 add() 메소드를 이용해 순서대로 Layer 를 하나씩 추가.
model = keras.Sequential()  

model.add(keras.layers.InputLayer((28, 28))) 

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256))
model.add(keras.layers.ReLU())

model.add(keras.layers.Dense(units=128))
model.add(keras.layers.ReLU())
model.add(keras.layers.Dense(units=10))

model.add(keras.layers.Softmax(name='output'))

In [ ]:
# Sequential Model 정의 2: 객체 생성시 Layer들을 순서대로 리스트로 묶어 전달.
model2 = keras.Sequential([
    keras.layers.InputLayer((28,28)), 
    keras.layers.Flatten(),
    keras.layers.Dense(256),
    keras.layers.ReLU(),
    keras.layers.Dense(units=128),
    keras.layers.ReLU(),
    keras.layers.Dense(10),
    keras.layers.Softmax()
])


In [ ]:
# 모델의 구조를 확인
model.summary()

In [ ]:
# graphviz를 이용한 모델구조를 시각화 
## 다음 패키지를 설치해야한다.
## pip install graphviz pydot pydotplus
keras.utils.plot_model(model, 
                       show_shapes=True 
                       , to_file='model_shapes.png'
                      )

## 컴파일 단계
- 정의된 딥러닝 모델을 학습할 수 있는 상태로 만들어 주기 위해 다음을 추가적으로 설정 한다.
    - Optimizer
    - 손실함수
    - 평가지표

In [ ]:
model.compile(optimizer='adam', 
             loss='categorical_crossentropy', 
             metrics=['accuracy']
             )

## 학습 (fit)
- model.fit()
    - 모델 학습 메소드. 
    - 학습과정의 Log를 **History** 객체에 넣어 반환한다.
- **History**: train 시 에폭별 평가지표값들을 모아서 제공. 


In [ ]:
# 모델 학습
history = model.fit(X_train, y_train,
                      epochs=10,
                      batch_size=100,
                      validation_split=0.3
                     )

##### History 시각화

In [ ]:
print(type(history))
print("학습 에폭리스트: ", history.epoch)
print("에폭수, 에폭당 step수: ", history.params)

In [ ]:
history.history

In [ ]:
# 학습 에폭별 loss와 accuracy 변화량 시각화
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'malgun gothic'
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(15, 6))
plt.suptitle('학습 평가 결과')

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.title('Accuracy')

plt.legend()
plt.tight_layout()
plt.show()

## 테스트셋 평가

In [ ]:
result = model.evaluate(X_test, y_test)

In [ ]:
result

## 새로운 데이터 추론
- 새로운 데이터를 추론하기 전에 학습데이터에 했던 전처리과정을 동일하게 적용 한 뒤 추론한다.

### 추론 메소드
- predict()
    - **분류:** 각 클래스 별 확률 반환
    - **회귀:** 최종 예측 결과
- 분류문제일때 predict() 결과에서 class label 출력하기    
    - 이진 분류(binary classification)
        - `numpy.where(model.predict(x) > 0.5, 1, 0).astype("int32")`
    - 다중클래스 분류(multi-class classification)
        - `numpy.argmax(model.predict(x), axis=1)`

In [ ]:
X_new = X_test[:3]
X_new.shape

In [ ]:
pred = model.predict(X_new)

In [ ]:
print(pred.shape)
np.round(pred, 3)

In [ ]:
np.argmax(pred, axis=1)

In [ ]:
np.argmax(y_test[:3], axis=-1)